In [11]:
import pandas as pd
import numpy as np
from pathlib import Path
import librosa
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from algorithms import encode_columns, svm_model, accuracy_calculator, random_forest_model, array_column_spread
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [12]:
df = pd.read_csv("dsl_data/development.csv")
df_eval = pd.read_csv("dsl_data/evaluation.csv")
y = df.values[:,3:5].sum(axis=1)
df2_eval = pd.read_csv("x_eval.csv")
df2= pd.read_csv("x.csv")

In [13]:
# def add_mean_column(df, prefix):
#     mean_column = df.filter(regex=f"^{prefix}.*").mean(axis=1)
#     df[prefix + "_mean"] = mean_column

# add_mean_column(df, 'chroma')

In [14]:
df2.columns

Index(['Unnamed: 0', 'Self-reported fluency level ', 'First Language spoken',
       'speakerId', 'gender', 'ageRange', 'chroma_0', 'chroma_1', 'chroma_2',
       'chroma_3', 'chroma_4', 'chroma_5', 'chroma_6', 'chroma_7', 'chroma_8',
       'chroma_9', 'chroma_10', 'chroma_11', 'tonnetz_0', 'tonnetz_1',
       'tonnetz_2', 'tonnetz_3', 'tonnetz_4', 'tonnetz_5',
       'spectral_contrast_0', 'spectral_contrast_1', 'spectral_contrast_2',
       'spectral_contrast_3', 'spectral_contrast_4', 'spectral_contrast_5',
       'spectral_contrast_6', 'rmse', 'sf', 'spectral_rolloff',
       'zero_crossing_rate', 'mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4',
       'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11',
       'mfcc_12', 'mfcc_13', 'mfcc_14', 'mfcc_15', 'mfcc_16', 'mfcc_17',
       'mfcc_18', 'mfcc_19', 'mfcc_20', 'mfcc_21', 'mfcc_22', 'mfcc_23',
       'mfcc_24', 'mfcc_25', 'mfcc_26', 'mfcc_27', 'mfcc_28', 'mfcc_29',
       'mfcc_30', 'mfcc_31', 'mfcc_32', 'm

In [19]:
features = [
       'Self-reported fluency level ', 'First Language spoken',
       'speakerId', 'gender', 'ageRange', 'tonnetz_0', 'tonnetz_1',
       'tonnetz_2', 'tonnetz_3', 'tonnetz_4', 'tonnetz_5',
       'spectral_contrast_0', 'spectral_contrast_1', 'spectral_contrast_2',
       'spectral_contrast_3', 'spectral_contrast_4', 'spectral_contrast_5',
       'spectral_contrast_6', 'rmse', 'sf', 'spectral_rolloff',
       'zero_crossing_rate', 'chroma_mean','mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4',
       'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11',
       'mfcc_12', 'mfcc_13', 'mfcc_14', 'mfcc_15', 'mfcc_16', 'mfcc_17',
       'mfcc_18', 'mfcc_19', 'mfcc_20', 'mfcc_21', 'mfcc_22', 'mfcc_23',
       'mfcc_24', 'mfcc_25', 'mfcc_26', 'mfcc_27', 'mfcc_28', 'mfcc_29',
       'mfcc_30', 'mfcc_31', 'mfcc_32', 'mfcc_33', 'mfcc_34', 'mfcc_35',
       'mfcc_36', 'mfcc_37', 'mfcc_38', 'mfcc_39', 'mfcc_40', 'mfcc_41',
       'mfcc_42', 'mfcc_43', 'mfcc_44', 'mfcc_45', 'mfcc_46', 'mfcc_47',
       'mfcc_48', 'mfcc_49'
]

In [6]:
X_eval = ['Self-reported fluency level ', 'First Language spoken',
       'speakerId', 'gender', 'ageRange', 'chroma_0', 'chroma_1', 'chroma_2',
       'chroma_3', 'chroma_4', 'chroma_5', 'chroma_6', 'chroma_7', 'chroma_8',
       'chroma_9', 'chroma_10', 'chroma_11', 'tonnetz_0', 'tonnetz_1',
       'tonnetz_2', 'tonnetz_3', 'tonnetz_4', 'tonnetz_5',
       'spectral_contrast_0', 'spectral_contrast_1', 'spectral_contrast_2',
       'spectral_contrast_3', 'spectral_contrast_4', 'spectral_contrast_5',
       'spectral_contrast_6', 'rmse', 'sf', 'spectral_rolloff',
       'zero_crossing_rate', 'mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4',
       'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11',
       'mfcc_12', 'mfcc_13', 'mfcc_14', 'mfcc_15', 'mfcc_16', 'mfcc_17',
       'mfcc_18', 'mfcc_19', 'mfcc_20', 'mfcc_21', 'mfcc_22', 'mfcc_23',
       'mfcc_24', 'mfcc_25', 'mfcc_26', 'mfcc_27', 'mfcc_28', 'mfcc_29',
       'mfcc_30', 'mfcc_31', 'mfcc_32', 'mfcc_33', 'mfcc_34', 'mfcc_35',
       'mfcc_36', 'mfcc_37', 'mfcc_38', 'mfcc_39', 'mfcc_40', 'mfcc_41',
       'mfcc_42', 'mfcc_43', 'mfcc_44', 'mfcc_45', 'mfcc_46', 'mfcc_47',
       'mfcc_48', 'mfcc_49', 'chroma_mean', 'tonnetz_mean',
       'spectral_contrast_mean', 'mfcc_mean']

In [20]:
X = df2[features].copy()

len(X.columns)

73

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

# Create an instance of the RandomForestClassifier
clf = RandomForestClassifier()

# Define the parameter grid for the RandomizedSearchCV
param_grid = {
    "n_estimators": sp_randint(50, 200),
    "max_depth": sp_randint(2, 10),
    "min_samples_split": sp_randint(2, 10),
    "min_samples_leaf": sp_randint(1, 10),
    "max_features": ['auto', 'sqrt', 'log2']
}

# Create an instance of the RandomizedSearchCV
random_search = RandomizedSearchCV(clf, param_distributions=param_grid, n_iter=100, cv=4)

# Fit the randomized search on the data
random_search.fit(X, y)

# Print the best parameters
print("Best parameters: ", random_search.best_params_)

In [10]:
y_pred = svm_model(X_train, y_train, X_test)
accuracy_calculator(y_test, y_pred)

In [18]:
y_pred = random_forest_model(X_train, X_test, y_train)
accuracy_calculator(y_test, y_pred)

0.3485540334855403

In [ ]:
## Implementation using k-fold
from sklearn.model_selection import KFold, cross_val_score
# define the number of folds and whether to shuffle the data
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# create an instance of the SVM model
clf = SVC()

# use cross_val_score function to perform k-fold cross-validation
scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

# print the mean accuracy and standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=0)

# shuffle the data before performing k-fold cross validation
X, y = shuffle(X, y)

# perform k-fold cross validation with 5 folds
scores = cross_val_score(clf, X, y, cv=5)

# calculate the mean accuracy of the model across all folds
accuracy = np.mean(scores)
print("Accuracy:", accuracy)
